# Tests

In [75]:
#from plot_prop import plot_direction, plot_pointcloud, plot_segments, plot_projection, plot_all_projections
from prop_info import extreme_points, vect_blade, d_blade
from get_segments import blade_alone, get_segments_points, get_planes
from major_axis import get_major_axis
from projections import all_projections
from myMathFunction import distance_point_plane
import pandas as pd
import numpy as np

ImportError: cannot import name 'all_projections'

In [71]:
propeller_coords = pd.read_csv('propeller_data.csv')

max_point, min_point, middle_point, highest_point, lowest_point = extreme_points(propeller_coords)

vect_length = vect_blade(max_point, min_point) 

dmiddle, dhighest, dlowest = d_blade(vect_length, middle_point, highest_point, lowest_point)

upper_blade, lower_blade = blade_alone(propeller_coords, vect_length, middle_point, dmiddle)

vect_out, vect_side = get_major_axis(propeller_coords, middle_point, vect_length)

nb_seg = 5
planes = get_planes(upper_blade, dmiddle, dhighest, vect_length, nb_seg)

segments = get_segments_points(upper_blade, planes, nb_seg)

C:\Users\Pauli\Desktop\PropellerProject\get_segments.py:48: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  segments["points"].append(blade.loc[index_segment].copy().as_matrix())


In [72]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

def plot_all_projections(proj_up, proj_down):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    for elem in proj_down:
        ax.plot(proj_down[elem]["X"], proj_down[elem]["Y"], proj_down[elem]["Z"], 'co', markersize=3, alpha=0.2)
    for elem in proj_up:
        ax.plot(proj_down[elem]["X"], proj_down[elem]["Y"], proj_down[elem]["Z"], 'ro', markersize=3, alpha=0.2)
    ax.set_xlabel('x_values', fontsize=15)
    ax.set_ylabel('y_values', fontsize=15)
    ax.set_zlabel('z_values', fontsize=15)
    plt.title('Projections on plane', fontsize=20)
    plt.show()

In [65]:
plan = planes[1]
segment_down = segments['points'][0]
segment_up = segments['points'][1]
nb_point = 500

In [66]:
proj_down = {}
proj_up = {}

#Plane i is associated with segments i-1 and i
for proj in range(1, nb_seg):
    df_d, df_u = project_on_plane(planes[proj], segments['points'][proj-1], segments['points'][proj], nb_point)
    proj_down[proj] = df_d
    proj_up[proj] = df_u

In [67]:
def interpole_on_plane(plan, segment_down, segment_up, nb_point):
    distance_down = []
    distance_up = []
    
    # Get the distance of the upper and lower point to the plan
    for point in segment_down:
        distance_down.append(distance_point_plane(point, plan)) 
    for point in segment_up:
        distance_up.append(distance_point_plane(point, plan)) 
    
    # Get the 100 point closest to plane on each side
    sort_idx_down = np.argsort(distance_down)
    sort_idx_up   = np.argsort(distance_up)
    
    sort_idx_down = sort_idx_down[:nb_point]
    sort_idx_up   = sort_idx_up[:nb_point]
    
    point_down = segment_down[sort_idx_down]
    point_up   = segment_up[sort_idx_up]
    
    df_d = pd.DataFrame(point_down)
    df_d.columns = ['X', 'Y','Z']
    df_u = pd.DataFrame(point_up)
    df_u.columns = ['X', 'Y','Z']
    '''
    # Project point on plan
    pp_d = []
    pp_u = []
    for p in point_down:
        pp_d.append(project_point(p, plan))
    for p in point_up:
        pp_u.append(project_point(p, plan))
        
    df_d = pd.DataFrame(pp_d)
    df_d.columns = ['X', 'Y','Z']
    df_u = pd.DataFrame(pp_u)
    df_u.columns = ['X', 'Y','Z']
    '''
    return df_d, df_u

In [68]:
df_d, df_u = interpole_on_plane(plan, segment_down, segment_up, nb_point)

NameError: name 'distance_point_plane' is not defined